In [ ]:
#downloading necessaries libs
!pip install pennylane
!pip install jax
!pip install jaxopt

In [129]:
#importing necessaries libs
import pennylane as qml
from jax import numpy as jnp
import jax
import random
import numpy as np

In [130]:
#I have decided to create q-node for 4 qubits
n_qubits=4
#lightning.qubit is used for simulating the circuit
dev1 = qml.device("lightning.qubit", wires=n_qubits)

In [131]:
#new random circuit
@qml.qnode(dev1)
def circuit(params):
    for i in range(n_qubits):
        qml.RX(params[0][i], wires=i)
        qml.RY(params[1][i], wires=i)
        qml.RZ(params[2][i], wires=i)

    qml.CNOT(wires=[0,3])
    qml.CNOT(wires=[0,2])
    qml.CNOT(wires=[0,1])
    qml.CNOT(wires=[3,0])
    #expected val with Z oper for all qubits
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

In [132]:
# Define a target vector for the cost function
target = jnp.array([1.0, -1.0, 1.0, -1.0])

In [133]:
# Define the cost function
def cost(params):
    # Ensure params is a JAX array
    params = jnp.array(params)
    # Compute the circuit output
    circuit_output = circuit(params)
    # Ensure circuit_output is a JAX array
    circuit_output = jnp.array(circuit_output)
    # Compute the cost as the mean squared error
    return jnp.mean((circuit_output - target) ** 2)

In [134]:
# Define a function to compute the gradient of the circuit
def gradient_function1(params_batch):
    # Compute the gradient of the circuit output with respect to the parameters
    return jax.grad(lambda p: cost(p))(params_batch)

In [135]:
# Define a function to compute the gradient of the circuit
def gradient_function(params_batch):
    # Compute the gradient of the circuit output with respect to the parameters
    return jax.grad(lambda p: jnp.sum(jnp.array(circuit(p))))(params_batch)

In [136]:
#init 3 random 4-qubits
params = np.random.rand(3,n_qubits)
result=circuit(params)
print(params)
print(result)

[[0.62256124 0.59435833 0.561718   0.28506937]
 [0.76276722 0.6054833  0.35354627 0.8369957 ]
 [0.73406615 0.62008058 0.50773193 0.32091469]]
[0.6426694109604557, 0.4000793572807899, 0.4663111774774553, 0.37743769929915527]


In [137]:
dcircuit = gradient_function(params)
print(dcircuit)

[[-8.92798424e-01 -2.70408183e-01 -2.93470800e-01 -2.98943341e-01]
 [-1.18876696e+00 -2.76941717e-01 -1.72093317e-01 -1.13120830e+00]
 [ 7.84469042e-17 -1.06795395e-17 -2.45773161e-17  0.00000000e+00]]


In [138]:
init_params = np.array([
    [0.1, 0.2, 0.3, 0.4],  # Parameters for the first set of 4 qubits
    [0.5, 0.6, 0.7, 0.8],  # Parameters for the second set of 4 qubits
    [0.9, 1.0, 1.1, 1.2]   # Parameters for the third set of 4 qubits
])
print(cost(init_params))

1.401392


In [139]:
import jaxopt

# initialise the optimizer
opt = jaxopt.GradientDescent(cost, stepsize=0.4, acceleration = False)

# set the number of steps
steps = 100
# set the initial parameter values
params = init_params
opt_state = opt.init_state(params)

for i in range(steps):
    # update the circuit parameters
    params, opt_state = opt.update(params, opt_state)

    if (i + 1) % 5 == 0:
        print("Cost after step {:5d}: {: .7f}".format(i + 1, cost(params)))

print("Optimized rotation angles: {}".format(params))

Cost after step     5:  0.8423170
Cost after step    10:  0.7831995
Cost after step    15:  0.7499260
Cost after step    20:  0.7078516
Cost after step    25:  0.6481324
Cost after step    30:  0.5870115
Cost after step    35:  0.5434134
Cost after step    40:  0.5203503
Cost after step    45:  0.5100675
Cost after step    50:  0.5055485
Cost after step    55:  0.5033940
Cost after step    60:  0.5022542
Cost after step    65:  0.5015933
Cost after step    70:  0.5011806
Cost after step    75:  0.5009076
Cost after step    80:  0.5007183
Cost after step    85:  0.5005820
Cost after step    90:  0.5004809
Cost after step    95:  0.5004038
Cost after step   100:  0.5003438
Optimized rotation angles: [[0.04230545 0.05970023 0.02843451 0.5881239 ]
 [0.19156484 2.9830647  0.06170619 1.5581872 ]
 [0.9        1.         1.1        1.2       ]]


In [ ]:
#converging after 70 steps